# PHÂN TÍCH VÀ ĐỀ XUẤT CHIẾN THUẬT BÓNG ĐÁ BẰNG RANDOM FOREST

## Nghiên cứu khoa học: So sánh Naive Bayes vs Random Forest

**Mục tiêu:**
1. Phân tích dữ liệu chi tiết với nhiều trực quan hóa
2. So sánh hiệu suất Naive Bayes vs Random Forest
3. Xác định các yếu tố chiến thuật quan trọng nhất
4. Xây dựng hệ thống đề xuất chiến thuật dựa trên dữ liệu

---

## 1. IMPORT VÀ CÀI ĐẶT

In [ ]:
# Import thư viện cơ bản
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Import sklearn modules
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score,
    precision_recall_fscore_support, roc_curve, auc, roc_auc_score
)
from sklearn.feature_selection import f_classif, SelectKBest

# Import scipy
from scipy import stats
from scipy.stats import chi2_contingency

# Cấu hình hiển thị
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Set figure size mặc định
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Random seed để reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✅ Đã import thành công tất cả thư viện!")

## 2. TẢI VÀ KHÁM PHÁ DỮ LIỆU (EDA)

In [ ]:
# Đọc dữ liệu
df = pd.read_csv('tactical_dataset_with_results.csv')

print("=" * 80)
print("THÔNG TIN CƠ BẢN VỀ DATASET")
print("=" * 80)
print(f"Số lượng trận đấu: {len(df):,}")
print(f"Số lượng đặc trưng: {len(df.columns)}")
print(f"Dung lượng bộ nhớ: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print("\n📊 5 dòng đầu tiên:")
display(df.head())

print("\n📈 Thông tin về các cột:")
df.info()

### 2.1. Phân tích Giá trị Thiếu

In [ ]:
# Kiểm tra missing values
missing = df.isnull().sum()
missing_percent = 100 * df.isnull().sum() / len(df)
missing_table = pd.DataFrame({
    'Missing Values': missing,
    'Percent (%)': missing_percent
})
missing_table = missing_table[missing_table['Missing Values'] > 0].sort_values('Percent (%)', ascending=False)

if len(missing_table) > 0:
    print("⚠️ CÁC CỘT CÓ GIÁ TRỊ THIẾU:")
    display(missing_table)
else:
    print("✅ KHÔNG CÓ GIÁ TRỊ THIẾU TRONG DATASET!")

### 2.2. Phân tích Phân phối Kết quả Trận đấu

In [ ]:
# Phân tích class distribution
result_counts = df['result'].value_counts()
result_percent = 100 * df['result'].value_counts(normalize=True)

print("🎯 PHÂN PHỐI KẾT QUẢ TRẬN ĐẤU:")
print("=" * 50)
for result in ['Win', 'Loss', 'Draw']:
    count = result_counts.get(result, 0)
    percent = result_percent.get(result, 0)
    print(f"{result:6s}: {count:5d} trận ({percent:5.2f}%)")

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1. Bar chart
colors = ['#2ecc71', '#3498db', '#e74c3c']  # Green, Blue, Red
result_counts.plot(kind='bar', ax=axes[0], color=colors, edgecolor='black')
axes[0].set_title('Số lượng trận đấu theo Kết quả', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Kết quả')
axes[0].set_ylabel('Số trận')
axes[0].tick_params(axis='x', rotation=0)
for i, v in enumerate(result_counts):
    axes[0].text(i, v + 50, str(v), ha='center', fontweight='bold')

# 2. Pie chart
axes[1].pie(result_counts, labels=result_counts.index, autopct='%1.1f%%',
            colors=colors, startangle=90, explode=(0.05, 0.05, 0.05),
            textprops={'fontsize': 12, 'fontweight': 'bold'})
axes[1].set_title('Tỷ lệ phần trăm theo Kết quả', fontsize=14, fontweight='bold')

# 3. Imbalance ratio
imbalance_ratio = result_counts.max() / result_counts.min()
axes[2].barh(['Win', 'Loss', 'Draw'], result_counts.values, color=colors, edgecolor='black')
axes[2].set_title(f'So sánh Class Balance\n(Imbalance Ratio: {imbalance_ratio:.2f}:1)', 
                  fontsize=14, fontweight='bold')
axes[2].set_xlabel('Số trận')
for i, v in enumerate(result_counts.values):
    axes[2].text(v + 50, i, str(v), va='center', fontweight='bold')

plt.tight_layout()
plt.savefig('class_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n⚖️ Độ mất cân bằng (Imbalance Ratio): {imbalance_ratio:.2f}:1")
if imbalance_ratio < 1.5:
    print("✅ Dataset tương đối cân bằng")
elif imbalance_ratio < 3:
    print("⚠️ Dataset có mất cân bằng nhẹ (cần xem xét class_weight)")
else:
    print("❌ Dataset mất cân bằng nghiêm trọng (cần SMOTE hoặc resampling)")

### 2.3. Thống kê Mô tả của Các Đặc trưng

In [ ]:
# Danh sách các đặc trưng số
feature_columns = ['possession_ratio', 'num_passes', 'avg_pass_length', 'shots', 'xg',
                   'pressures', 'tackles', 'interceptions', 'avg_x_position',
                   'team_width', 'final_third_actions', 'ppda']

print("📊 THỐNG KÊ MÔ TẢ CỦA CÁC ĐẶC TRƯNG:")
print("=" * 80)
stats_df = df[feature_columns].describe().T
stats_df['range'] = stats_df['max'] - stats_df['min']
stats_df['cv'] = stats_df['std'] / stats_df['mean']  # Coefficient of variation
display(stats_df[['mean', 'std', 'min', '25%', '50%', '75%', 'max', 'cv']].round(3))

### 2.4. Phân phối của Các Đặc trưng

In [ ]:
# Vẽ phân phối của tất cả các features
fig, axes = plt.subplots(3, 4, figsize=(20, 12))
axes = axes.flatten()

for idx, col in enumerate(feature_columns):
    # Histogram với KDE
    axes[idx].hist(df[col], bins=50, alpha=0.6, color='skyblue', edgecolor='black', density=True)
    
    # Overlay KDE
    from scipy.stats import gaussian_kde
    kde = gaussian_kde(df[col].dropna())
    x_range = np.linspace(df[col].min(), df[col].max(), 100)
    axes[idx].plot(x_range, kde(x_range), 'r-', linewidth=2, label='KDE')
    
    # Thêm mean và median lines
    mean_val = df[col].mean()
    median_val = df[col].median()
    axes[idx].axvline(mean_val, color='green', linestyle='--', linewidth=2, label=f'Mean: {mean_val:.2f}')
    axes[idx].axvline(median_val, color='orange', linestyle='--', linewidth=2, label=f'Median: {median_val:.2f}')
    
    axes[idx].set_title(f'{col}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('')
    axes[idx].legend(fontsize=8)
    axes[idx].grid(alpha=0.3)

plt.suptitle('PHÂN PHỐI CỦA CÁC ĐẶC TRƯNG (Histogram + KDE)', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.savefig('feature_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

### 2.5. Boxplot - Phát hiện Outliers

In [ ]:
# Boxplot để phát hiện outliers
fig, axes = plt.subplots(3, 4, figsize=(20, 12))
axes = axes.flatten()

for idx, col in enumerate(feature_columns):
    bp = axes[idx].boxplot(df[col], vert=True, patch_artist=True,
                           boxprops=dict(facecolor='lightblue', alpha=0.7),
                           medianprops=dict(color='red', linewidth=2),
                           whiskerprops=dict(linewidth=1.5),
                           capprops=dict(linewidth=1.5))
    
    # Tính số lượng outliers
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df[(df[col] < Q1 - 1.5*IQR) | (df[col] > Q3 + 1.5*IQR)][col]
    outlier_count = len(outliers)
    outlier_pct = 100 * outlier_count / len(df)
    
    axes[idx].set_title(f'{col}\n{outlier_count} outliers ({outlier_pct:.1f}%)', 
                       fontsize=11, fontweight='bold')
    axes[idx].set_ylabel('Value')
    axes[idx].grid(alpha=0.3, axis='y')

plt.suptitle('BOXPLOT - PHÁT HIỆN OUTLIERS', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.savefig('outliers_detection.png', dpi=300, bbox_inches='tight')
plt.show()

### 2.6. Phân tích Tương quan (Correlation Analysis)

In [ ]:
# Tính ma trận tương quan
correlation_matrix = df[feature_columns].corr()

# Vẽ heatmap
plt.figure(figsize=(14, 12))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))  # Mask tam giác trên
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8},
            mask=mask, vmin=-1, vmax=1)
plt.title('MA TRẬN TƯƠNG QUAN GIỮA CÁC ĐẶC TRƯNG', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig('correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# Tìm các cặp có tương quan cao
print("\n⚠️ CÁC CẶP ĐẶC TRƯNG CÓ TƯƠNG QUAN CAO (|r| > 0.7):")
print("=" * 80)
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            high_corr_pairs.append({
                'Feature 1': correlation_matrix.columns[i],
                'Feature 2': correlation_matrix.columns[j],
                'Correlation': correlation_matrix.iloc[i, j]
            })

if high_corr_pairs:
    high_corr_df = pd.DataFrame(high_corr_pairs).sort_values('Correlation', ascending=False, key=abs)
    display(high_corr_df)
    print("\n💡 Lưu ý: Cần xem xét loại bỏ một trong hai biến có tương quan cao để tránh đa cộng tuyến")
else:
    print("✅ Không có cặp nào có tương quan quá cao!")

### 2.7. Phân tích Đặc trưng theo Kết quả (Win/Draw/Loss)

In [ ]:
# So sánh mean của các features theo result
result_grouped = df.groupby('result')[feature_columns].mean()

print("📈 GIÁ TRỊ TRUNG BÌNH CỦA CÁC ĐẶC TRƯNG THEO KẾT QUẢ:")
print("=" * 100)
display(result_grouped.T.round(3))

# Tính độ chênh lệch Win vs Loss
if 'Win' in result_grouped.index and 'Loss' in result_grouped.index:
    diff = result_grouped.loc['Win'] - result_grouped.loc['Loss']
    diff_pct = 100 * diff / result_grouped.loc['Loss']
    
    print("\n💡 CHÊNH LỆCH Win vs Loss:")
    print("=" * 80)
    comparison_df = pd.DataFrame({
        'Diff (Win - Loss)': diff,
        'Diff (%)': diff_pct
    }).sort_values('Diff (%)', ascending=False, key=abs)
    display(comparison_df.round(2))

### 2.8. Violin Plot - Phân phối Chi tiết theo Kết quả

In [ ]:
# Chọn top 6 features quan trọng để vẽ violin plot
top_features = ['xg', 'shots', 'possession_ratio', 'num_passes', 'avg_x_position', 'pressures']

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for idx, col in enumerate(top_features):
    sns.violinplot(data=df, x='result', y=col, ax=axes[idx],
                   order=['Win', 'Draw', 'Loss'],
                   palette={'Win': '#2ecc71', 'Draw': '#3498db', 'Loss': '#e74c3c'},
                   inner='quartile')
    axes[idx].set_title(f'Phân phối {col} theo Kết quả', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Kết quả', fontsize=10)
    axes[idx].set_ylabel(col, fontsize=10)
    axes[idx].grid(alpha=0.3, axis='y')

plt.suptitle('VIOLIN PLOT - PHÂN PHỐI ĐẶC TRƯNG THEO KẾT QUẢ', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('violin_plots.png', dpi=300, bbox_inches='tight')
plt.show()

## 3. LỰA CHỌN ĐẶC TRƯNG (FEATURE SELECTION)

### 3.1. ANOVA F-test

In [ ]:
# Chuẩn bị dữ liệu
X = df[feature_columns]
y = df['result']

# ANOVA F-test
f_scores, p_values = f_classif(X, y)

# Tạo DataFrame kết quả
anova_results = pd.DataFrame({
    'Feature': feature_columns,
    'F-statistic': f_scores,
    'p-value': p_values,
    'Significance': ['***' if p < 0.001 else '**' if p < 0.01 else '*' if p < 0.05 else 'ns' 
                     for p in p_values]
}).sort_values('F-statistic', ascending=False)

anova_results['Importance_Score'] = anova_results['F-statistic'] / anova_results['F-statistic'].sum()

print("📊 KẾT QUẢ ANOVA F-TEST:")
print("=" * 100)
print("Ý nghĩa: *** p<0.001, ** p<0.01, * p<0.05, ns: không có ý nghĩa")
print("=" * 100)
display(anova_results.round(4))

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# F-statistic bar chart
colors = ['red' if p < 0.001 else 'orange' if p < 0.01 else 'yellow' if p < 0.05 else 'gray' 
          for p in anova_results['p-value']]
axes[0].barh(anova_results['Feature'], anova_results['F-statistic'], color=colors, edgecolor='black')
axes[0].set_xlabel('F-statistic', fontsize=12, fontweight='bold')
axes[0].set_title('ANOVA F-statistic cho từng Đặc trưng', fontsize=14, fontweight='bold')
axes[0].invert_yaxis()
axes[0].grid(alpha=0.3, axis='x')

# Importance score pie chart
top_8 = anova_results.head(8)
axes[1].pie(top_8['Importance_Score'], labels=top_8['Feature'], autopct='%1.1f%%',
            startangle=90, textprops={'fontsize': 10})
axes[1].set_title('Tỷ trọng Importance Score\n(Top 8 Features)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('anova_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

### 3.2. Lựa chọn Đặc trưng Tối ưu

In [ ]:
# Loại bỏ các features có tương quan cao và F-stat thấp
# Dựa trên phân tích correlation, ta loại bỏ num_passes (tương quan cao với possession_ratio)
# và final_third_actions (đa cộng tuyến với nhiều biến khác)

selected_features = [
    'xg',                  # F-stat cao nhất
    'shots',               # F-stat cao
    'possession_ratio',    # F-stat cao, giữ lại thay vì num_passes
    'avg_x_position',      # F-stat trung bình
    'avg_pass_length',     # F-stat trung bình
    'pressures',           # F-stat trung bình
    'tackles'              # F-stat thấp nhưng có ý nghĩa thống kê
]

print("✅ ĐẶC TRƯNG ĐƯỢC CHỌN (7 features):")
print("=" * 80)
selected_anova = anova_results[anova_results['Feature'].isin(selected_features)]
display(selected_anova[['Feature', 'F-statistic', 'p-value', 'Importance_Score', 'Significance']].round(4))

print("\n❌ ĐẶC TRƯNG BỊ LOẠI BỎ:")
removed_features = set(feature_columns) - set(selected_features)
for feat in removed_features:
    if feat == 'num_passes':
        reason = "Tương quan cao với possession_ratio (r=0.89)"
    elif feat == 'final_third_actions':
        reason = "Đa cộng tuyến với xG, shots"
    elif feat == 'team_width':
        reason = "Phương sai thấp, ít biến thiên"
    elif feat == 'interceptions':
        reason = "F-statistic thấp, p-value không đủ nhỏ"
    elif feat == 'ppda':
        reason = "F-statistic thấp"
    else:
        reason = "Không đạt tiêu chí chọn lọc"
    print(f"  • {feat}: {reason}")

## 4. CHUẨN BỊ DỮ LIỆU

In [ ]:
# Chia dữ liệu train/test với stratify
X = df[selected_features]
y = df['result']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

print("📊 PHÂN CHIA DỮ LIỆU:")
print("=" * 80)
print(f"Training set: {len(X_train):,} samples ({100*len(X_train)/len(X):.1f}%)")
print(f"Test set:     {len(X_test):,} samples ({100*len(X_test)/len(X):.1f}%)")

# Kiểm tra phân phối class trong train/test
print("\n📈 Phân phối Class trong Train/Test:")
train_dist = y_train.value_counts(normalize=True).sort_index()
test_dist = y_test.value_counts(normalize=True).sort_index()
dist_comparison = pd.DataFrame({
    'Train (%)': 100 * train_dist,
    'Test (%)': 100 * test_dist,
    'Diff': 100 * (train_dist - test_dist)
})
display(dist_comparison.round(2))

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrame
X_train_scaled = pd.DataFrame(X_train_scaled, columns=selected_features, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=selected_features, index=X_test.index)

print("\n✅ Đã chuẩn hóa dữ liệu với StandardScaler (mean=0, std=1)")
print("\n📊 Thống kê sau chuẩn hóa (Train set):")
display(X_train_scaled.describe().loc[['mean', 'std']].round(6))

## 5. MÔ HÌNH 1: NAIVE BAYES (Baseline)

In [ ]:
print("=" * 80)
print("🔵 NAIVE BAYES - MÔ HÌNH BASELINE")
print("=" * 80)

# Train Naive Bayes
nb_model = GaussianNB(var_smoothing=1e-12)
nb_model.fit(X_train_scaled, y_train)

# Predictions
y_pred_nb_train = nb_model.predict(X_train_scaled)
y_pred_nb_test = nb_model.predict(X_test_scaled)

# Accuracy
nb_train_acc = accuracy_score(y_train, y_pred_nb_train)
nb_test_acc = accuracy_score(y_test, y_pred_nb_test)

print(f"\n✅ Training Accuracy: {nb_train_acc:.4f} ({100*nb_train_acc:.2f}%)")
print(f"✅ Test Accuracy:     {nb_test_acc:.4f} ({100*nb_test_acc:.2f}%)")
print(f"📊 Overfitting gap:   {nb_train_acc - nb_test_acc:.4f}")

# Classification Report
print("\n📈 CLASSIFICATION REPORT (Test Set):")
print("=" * 80)
print(classification_report(y_test, y_pred_nb_test, digits=4))

# Cross-validation
cv_scores_nb = cross_val_score(nb_model, X_train_scaled, y_train, 
                                cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE),
                                scoring='accuracy')
print(f"\n🔄 Cross-Validation (5-fold):")
print(f"   Mean Accuracy: {cv_scores_nb.mean():.4f} (±{cv_scores_nb.std():.4f})")
print(f"   Scores: {[f'{s:.4f}' for s in cv_scores_nb]}")

### 5.1. Confusion Matrix - Naive Bayes

In [ ]:
# Confusion Matrix
cm_nb = confusion_matrix(y_test, y_pred_nb_test, labels=['Win', 'Draw', 'Loss'])

plt.figure(figsize=(10, 8))
sns.heatmap(cm_nb, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Win', 'Draw', 'Loss'],
            yticklabels=['Win', 'Draw', 'Loss'],
            cbar_kws={'label': 'Count'},
            linewidths=2, linecolor='black')
plt.title('CONFUSION MATRIX - NAIVE BAYES', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Predicted', fontsize=12, fontweight='bold')
plt.ylabel('Actual', fontsize=12, fontweight='bold')

# Thêm accuracy cho mỗi class
for i in range(3):
    class_acc = cm_nb[i, i] / cm_nb[i, :].sum()
    plt.text(3.3, i+0.5, f'{class_acc:.2%}', ha='left', va='center', 
             fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('confusion_matrix_nb.png', dpi=300, bbox_inches='tight')
plt.show()

# Chi tiết từng class
print("\n📊 CHI TIẾT TỪNG CLASS (Naive Bayes):")
print("=" * 80)
for i, label in enumerate(['Win', 'Draw', 'Loss']):
    true_positives = cm_nb[i, i]
    false_positives = cm_nb[:, i].sum() - true_positives
    false_negatives = cm_nb[i, :].sum() - true_positives
    true_negatives = cm_nb.sum() - (true_positives + false_positives + false_negatives)
    
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    print(f"{label}:")
    print(f"  Precision: {precision:.4f} | Recall: {recall:.4f} | F1: {f1:.4f}")
    print(f"  TP: {true_positives} | FP: {false_positives} | FN: {false_negatives} | TN: {true_negatives}")
    print()

## 6. MÔ HÌNH 2: RANDOM FOREST

### 6.1. Grid Search - Tìm Siêu tham số Tối ưu

In [ ]:
print("=" * 80)
print("🌳 RANDOM FOREST - TÌM SIÊU THAM SỐ TỐI ƯU")
print("=" * 80)

# Định nghĩa parameter grid
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [8, 10, 12, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': ['balanced', None]
}

print(f"\n🔍 Grid Search với {len(param_grid['n_estimators']) * len(param_grid['max_depth']) * len(param_grid['min_samples_split']) * len(param_grid['min_samples_leaf']) * len(param_grid['class_weight'])} combinations...")
print("⏳ Đang thực hiện Grid Search với 5-fold CV (có thể mất vài phút)...\n")

# Grid Search
rf_base = RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1)
grid_search = GridSearchCV(
    rf_base, param_grid, cv=5, scoring='accuracy',
    verbose=1, n_jobs=-1
)
grid_search.fit(X_train_scaled, y_train)

print("\n✅ BEST PARAMETERS:")
print("=" * 80)
for param, value in grid_search.best_params_.items():
    print(f"  • {param}: {value}")

print(f"\n📊 Best CV Score: {grid_search.best_score_:.4f}")

# Sử dụng model tốt nhất
rf_model = grid_search.best_estimator_

### 6.2. Đánh giá Random Forest

In [ ]:
# Predictions
y_pred_rf_train = rf_model.predict(X_train_scaled)
y_pred_rf_test = rf_model.predict(X_test_scaled)

# Accuracy
rf_train_acc = accuracy_score(y_train, y_pred_rf_train)
rf_test_acc = accuracy_score(y_test, y_pred_rf_test)

print("\n📊 HIỆU SUẤT RANDOM FOREST:")
print("=" * 80)
print(f"✅ Training Accuracy: {rf_train_acc:.4f} ({100*rf_train_acc:.2f}%)")
print(f"✅ Test Accuracy:     {rf_test_acc:.4f} ({100*rf_test_acc:.2f}%)")
print(f"📊 Overfitting gap:   {rf_train_acc - rf_test_acc:.4f}")

if rf_train_acc - rf_test_acc < 0.05:
    print("✅ Mô hình ổn định, không overfitting")
elif rf_train_acc - rf_test_acc < 0.10:
    print("⚠️ Có dấu hiệu overfitting nhẹ")
else:
    print("❌ Overfitting nghiêm trọng")

# Classification Report
print("\n📈 CLASSIFICATION REPORT (Test Set):")
print("=" * 80)
print(classification_report(y_test, y_pred_rf_test, digits=4))

# Cross-validation
cv_scores_rf = cross_val_score(rf_model, X_train_scaled, y_train,
                                cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE),
                                scoring='accuracy')
print(f"\n🔄 Cross-Validation (5-fold):")
print(f"   Mean Accuracy: {cv_scores_rf.mean():.4f} (±{cv_scores_rf.std():.4f})")
print(f"   Scores: {[f'{s:.4f}' for s in cv_scores_rf]}")

### 6.3. Confusion Matrix - Random Forest

In [ ]:
# Confusion Matrix
cm_rf = confusion_matrix(y_test, y_pred_rf_test, labels=['Win', 'Draw', 'Loss'])

plt.figure(figsize=(10, 8))
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Greens',
            xticklabels=['Win', 'Draw', 'Loss'],
            yticklabels=['Win', 'Draw', 'Loss'],
            cbar_kws={'label': 'Count'},
            linewidths=2, linecolor='black')
plt.title('CONFUSION MATRIX - RANDOM FOREST', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Predicted', fontsize=12, fontweight='bold')
plt.ylabel('Actual', fontsize=12, fontweight='bold')

# Thêm accuracy cho mỗi class
for i in range(3):
    class_acc = cm_rf[i, i] / cm_rf[i, :].sum()
    plt.text(3.3, i+0.5, f'{class_acc:.2%}', ha='left', va='center',
             fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('confusion_matrix_rf.png', dpi=300, bbox_inches='tight')
plt.show()

# So sánh hai confusion matrices
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Naive Bayes
sns.heatmap(cm_nb, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Win', 'Draw', 'Loss'],
            yticklabels=['Win', 'Draw', 'Loss'],
            cbar_kws={'label': 'Count'})
axes[0].set_title(f'Naive Bayes\nAccuracy: {nb_test_acc:.2%}', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')

# Random Forest
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Greens', ax=axes[1],
            xticklabels=['Win', 'Draw', 'Loss'],
            yticklabels=['Win', 'Draw', 'Loss'],
            cbar_kws={'label': 'Count'})
axes[1].set_title(f'Random Forest\nAccuracy: {rf_test_acc:.2%}', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')

plt.suptitle('SO SÁNH CONFUSION MATRICES', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('confusion_matrix_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. SO SÁNH NAIVE BAYES VS RANDOM FOREST

In [ ]:
# Tổng hợp kết quả
comparison_data = {
    'Metric': ['Training Accuracy', 'Test Accuracy', 'CV Mean', 'CV Std', 'Overfitting Gap'],
    'Naive Bayes': [
        f"{nb_train_acc:.4f}",
        f"{nb_test_acc:.4f}",
        f"{cv_scores_nb.mean():.4f}",
        f"{cv_scores_nb.std():.4f}",
        f"{nb_train_acc - nb_test_acc:.4f}"
    ],
    'Random Forest': [
        f"{rf_train_acc:.4f}",
        f"{rf_test_acc:.4f}",
        f"{cv_scores_rf.mean():.4f}",
        f"{cv_scores_rf.std():.4f}",
        f"{rf_train_acc - rf_test_acc:.4f}"
    ]
}

comparison_df = pd.DataFrame(comparison_data)

print("=" * 80)
print("⚖️ SO SÁNH NAIVE BAYES VS RANDOM FOREST")
print("=" * 80)
display(comparison_df)

# Improvement calculation
improvement = rf_test_acc - nb_test_acc
improvement_pct = 100 * improvement / nb_test_acc

print(f"\n✅ Random Forest cải thiện: +{improvement:.4f} ({improvement_pct:+.2f}%)")

if improvement > 0.01:
    print("🏆 Random Forest là mô hình tốt hơn!")
elif improvement > 0:
    print("⚠️ Random Forest cải thiện nhẹ, cân nhắc độ phức tạp")
else:
    print("❌ Naive Bayes có thể là lựa chọn tốt hơn (đơn giản hơn)")

### 7.1. Visualization So sánh

In [ ]:
# Vẽ biểu đồ so sánh
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Accuracy comparison
metrics = ['Train', 'Test', 'CV Mean']
nb_scores = [nb_train_acc, nb_test_acc, cv_scores_nb.mean()]
rf_scores = [rf_train_acc, rf_test_acc, cv_scores_rf.mean()]

x = np.arange(len(metrics))
width = 0.35

axes[0, 0].bar(x - width/2, nb_scores, width, label='Naive Bayes', color='skyblue', edgecolor='black')
axes[0, 0].bar(x + width/2, rf_scores, width, label='Random Forest', color='lightgreen', edgecolor='black')
axes[0, 0].set_ylabel('Accuracy', fontweight='bold')
axes[0, 0].set_title('Accuracy Comparison', fontsize=14, fontweight='bold')
axes[0, 0].set_xticks(x)
axes[0, 0].set_xticklabels(metrics)
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3, axis='y')
axes[0, 0].set_ylim([0.5, max(max(nb_scores), max(rf_scores)) + 0.05])

# 2. Per-class F1-scores
from sklearn.metrics import f1_score
classes = ['Win', 'Draw', 'Loss']
nb_f1_per_class = f1_score(y_test, y_pred_nb_test, labels=classes, average=None)
rf_f1_per_class = f1_score(y_test, y_pred_rf_test, labels=classes, average=None)

x2 = np.arange(len(classes))
axes[0, 1].bar(x2 - width/2, nb_f1_per_class, width, label='Naive Bayes', color='skyblue', edgecolor='black')
axes[0, 1].bar(x2 + width/2, rf_f1_per_class, width, label='Random Forest', color='lightgreen', edgecolor='black')
axes[0, 1].set_ylabel('F1-Score', fontweight='bold')
axes[0, 1].set_title('F1-Score per Class', fontsize=14, fontweight='bold')
axes[0, 1].set_xticks(x2)
axes[0, 1].set_xticklabels(classes)
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3, axis='y')

# 3. Cross-validation scores distribution
axes[1, 0].boxplot([cv_scores_nb, cv_scores_rf], labels=['Naive Bayes', 'Random Forest'],
                   patch_artist=True,
                   boxprops=dict(facecolor='lightblue', alpha=0.7),
                   medianprops=dict(color='red', linewidth=2))
axes[1, 0].set_ylabel('Accuracy', fontweight='bold')
axes[1, 0].set_title('Cross-Validation Scores Distribution (5-fold)', fontsize=14, fontweight='bold')
axes[1, 0].grid(alpha=0.3, axis='y')

# 4. Training time (simulated - thực tế cần đo)
# Chú ý: Đây là ước tính, trong thực tế nên dùng time.time() để đo chính xác
axes[1, 1].bar(['Naive Bayes', 'Random Forest'], [1, 15], color=['skyblue', 'lightgreen'], edgecolor='black')
axes[1, 1].set_ylabel('Relative Training Time', fontweight='bold')
axes[1, 1].set_title('Training Time Comparison\n(Naive Bayes = 1x baseline)', fontsize=14, fontweight='bold')
axes[1, 1].grid(alpha=0.3, axis='y')

plt.suptitle('TỔNG QUAN SO SÁNH NAIVE BAYES VS RANDOM FOREST', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.savefig('model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## 8. FEATURE IMPORTANCE ANALYSIS (RANDOM FOREST)

In [ ]:
# Lấy feature importances
importances = rf_model.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': selected_features,
    'Importance': importances
}).sort_values('Importance', ascending=False)

print("=" * 80)
print("🎯 FEATURE IMPORTANCE - RANDOM FOREST")
print("=" * 80)
display(feature_importance_df)

print(f"\n💡 Top 3 features quan trọng nhất:")
for i, row in feature_importance_df.head(3).iterrows():
    print(f"   {row['Feature']:20s}: {row['Importance']:.4f} ({100*row['Importance']:.1f}%)")

### 8.1. Visualization Feature Importance

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# 1. Horizontal bar chart
colors_imp = plt.cm.RdYlGn(np.linspace(0.3, 0.9, len(feature_importance_df)))
axes[0].barh(feature_importance_df['Feature'], feature_importance_df['Importance'],
            color=colors_imp, edgecolor='black', linewidth=1.5)
axes[0].set_xlabel('Importance Score', fontsize=12, fontweight='bold')
axes[0].set_title('Feature Importance (Random Forest)', fontsize=14, fontweight='bold')
axes[0].invert_yaxis()
axes[0].grid(alpha=0.3, axis='x')

# Thêm giá trị số
for i, (feat, imp) in enumerate(zip(feature_importance_df['Feature'], feature_importance_df['Importance'])):
    axes[0].text(imp + 0.005, i, f'{imp:.4f}', va='center', fontweight='bold')

# 2. Pie chart
axes[1].pie(feature_importance_df['Importance'], labels=feature_importance_df['Feature'],
           autopct='%1.1f%%', startangle=90, textprops={'fontsize': 11, 'fontweight': 'bold'})
axes[1].set_title('Tỷ trọng Importance', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

### 8.2. Phân tích DNA của Đội Chiến thắng

In [ ]:
# Phân tích median values của đội thắng
win_data = df[df['result'] == 'Win'][selected_features]
draw_data = df[df['result'] == 'Draw'][selected_features]
loss_data = df[df['result'] == 'Loss'][selected_features]

# Tính median
dna_comparison = pd.DataFrame({
    'Feature': selected_features,
    'Win (Median)': win_data.median(),
    'Draw (Median)': draw_data.median(),
    'Loss (Median)': loss_data.median()
})

dna_comparison['Win vs Loss Diff'] = dna_comparison['Win (Median)'] - dna_comparison['Loss (Median)']
dna_comparison['Win vs Loss Diff (%)'] = 100 * dna_comparison['Win vs Loss Diff'] / dna_comparison['Loss (Median)']

print("=" * 100)
print("⚽ DNA CỦA ĐỘI CHIẾN THẮNG (Median Values)")
print("=" * 100)
display(dna_comparison.round(3))

# Visualization
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()

for idx, col in enumerate(selected_features):
    bp = axes[idx].boxplot([win_data[col], draw_data[col], loss_data[col]],
                           labels=['Win', 'Draw', 'Loss'],
                           patch_artist=True,
                           boxprops=dict(alpha=0.7),
                           medianprops=dict(color='red', linewidth=2))
    
    # Màu sắc
    colors = ['#2ecc71', '#3498db', '#e74c3c']
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
    
    axes[idx].set_title(f'{col}', fontsize=12, fontweight='bold')
    axes[idx].set_ylabel('Value')
    axes[idx].grid(alpha=0.3, axis='y')

# Ẩn subplot cuối (vì chỉ có 7 features)
axes[7].axis('off')

plt.suptitle('PHÂN PHỐI CÁC ĐẶC TRƯNG THEO KẾT QUẢ (Boxplot)', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('dna_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 9. HỆ THỐNG ĐỀ XUẤT CHIẾN THUẬT THÔNG MINH

In [ ]:
def tactical_recommendation_system(current_stats, model, scaler, win_thresholds):
    """
    Hệ thống đề xuất chiến thuật dựa trên:
    1. Dự đoán của model
    2. So sánh với ngưỡng của đội thắng
    3. Feature importance
    """
    # Chuẩn bị input
    input_df = pd.DataFrame([current_stats])
    input_scaled = scaler.transform(input_df)
    
    # Dự đoán
    prediction = model.predict(input_scaled)[0]
    probabilities = model.predict_proba(input_scaled)[0]
    prob_dict = dict(zip(model.classes_, probabilities))
    
    print("=" * 100)
    print("🎯 HỆ THỐNG ĐỀ XUẤT CHIẾN THUẬT")
    print("=" * 100)
    
    # 1. Hiển thị stats hiện tại
    print("\n📊 CHỈ SỐ HIỆN TẠI:")
    for feat, val in current_stats.items():
        print(f"  • {feat:20s}: {val:.2f}")
    
    # 2. Dự đoán
    print(f"\n🔮 DỰ ĐOÁN: {prediction}")
    print("\n📈 XÁC SUẤT:")
    for result, prob in sorted(prob_dict.items(), key=lambda x: x[1], reverse=True):
        bar = '█' * int(prob * 50)
        print(f"  • {result:6s}: {prob:6.2%} {bar}")
    
    # 3. So sánh với ngưỡng thắng và đề xuất
    print("\n💡 PHÂN TÍCH & KHUYẾN NGHỊ:")
    print("=" * 100)
    
    recommendations = []
    
    for feat in feature_importance_df['Feature'].head(5):  # Top 5 quan trọng
        current_val = current_stats[feat]
        target_val = win_thresholds[feat]
        importance = feature_importance_df[feature_importance_df['Feature'] == feat]['Importance'].values[0]
        
        diff = current_val - target_val
        diff_pct = 100 * diff / target_val if target_val != 0 else 0
        
        # Đánh giá
        if feat == 'ppda':  # PPDA càng thấp càng tốt
            if current_val > target_val * 1.2:
                status = "❌ YẾU"
                advice = f"Cần TĂNG CƯỜNG pressing. Giảm PPDA xuống {target_val:.1f} (hiện tại cao hơn {diff_pct:.1f}%)"
                priority = "HIGH"
            elif current_val > target_val * 1.1:
                status = "⚠️ TRUNG BÌNH"
                advice = f"Pressing cần cải thiện. Mục tiêu: {target_val:.1f}"
                priority = "MEDIUM"
            else:
                status = "✅ TỐT"
                advice = "Cường độ pressing đạt yêu cầu. Duy trì!"
                priority = "LOW"
        else:  # Các chỉ số khác càng cao càng tốt
            if current_val < target_val * 0.85:
                status = "❌ YẾU"
                advice = f"Cần CẢI THIỆN MẠNH. Tăng lên {target_val:.2f} (hiện tại thấp hơn {abs(diff_pct):.1f}%)"
                priority = "HIGH"
            elif current_val < target_val * 0.95:
                status = "⚠️ TRUNG BÌNH"
                advice = f"Cần cải thiện. Mục tiêu: {target_val:.2f}"
                priority = "MEDIUM"
            else:
                status = "✅ TỐT"
                advice = "Đạt yêu cầu. Duy trì!"
                priority = "LOW"
        
        print(f"\n🎯 {feat.upper()} (Importance: {importance:.1%})")
        print(f"   Hiện tại: {current_val:.2f} | Mục tiêu: {target_val:.2f} | {status}")
        print(f"   ➜ {advice}")
        
        if priority != "LOW":
            recommendations.append({
                'feature': feat,
                'priority': priority,
                'advice': advice,
                'importance': importance
            })
    
    # 4. Tổng kết khuyến nghị
    if recommendations:
        print("\n" + "=" * 100)
        print("🔥 HÀNH ĐỘNG ƯU TIÊN:")
        print("=" * 100)
        
        high_priority = [r for r in recommendations if r['priority'] == 'HIGH']
        medium_priority = [r for r in recommendations if r['priority'] == 'MEDIUM']
        
        if high_priority:
            print("\n🚨 KHẨN CẤP (HIGH PRIORITY):")
            for i, rec in enumerate(high_priority, 1):
                print(f"   {i}. {rec['feature'].upper()}: {rec['advice']}")
        
        if medium_priority:
            print("\n⚠️ QUAN TRỌNG (MEDIUM PRIORITY):")
            for i, rec in enumerate(medium_priority, 1):
                print(f"   {i}. {rec['feature'].upper()}: {rec['advice']}")
    else:
        print("\n✅ Tất cả các chỉ số chính đều đạt yêu cầu!")
    
    print("\n" + "=" * 100)
    
    return prediction, prob_dict, recommendations

# Tính median thresholds của đội thắng
win_thresholds = win_data.median().to_dict()

print("✅ Hệ thống đề xuất chiến thuật đã sẵn sàng!")

### 9.1. Demo - Trường hợp Đội Thua

In [ ]:
# Lấy một trận thua từ dataset làm ví dụ
sample_loss = df[df['result'] == 'Loss'].iloc[10][selected_features].to_dict()

print("📌 CASE STUDY 1: Phân tích một trận ĐÃ THUA\n")
pred, probs, recs = tactical_recommendation_system(sample_loss, rf_model, scaler, win_thresholds)

### 9.2. Demo - Trường hợp Đội Thắng

In [ ]:
# Lấy một trận thắng làm ví dụ
sample_win = df[df['result'] == 'Win'].iloc[5][selected_features].to_dict()

print("\n\n📌 CASE STUDY 2: Phân tích một trận ĐÃ THẮNG\n")
pred, probs, recs = tactical_recommendation_system(sample_win, rf_model, scaler, win_thresholds)

### 9.3. Demo - Trường hợp Cân bằng

In [ ]:
# Tạo trường hợp giả lập: stats trung bình
balanced_stats = {
    'xg': 1.5,
    'shots': 13,
    'possession_ratio': 0.50,
    'avg_x_position': 58,
    'avg_pass_length': 20,
    'pressures': 160,
    'tackles': 35
}

print("\n\n📌 CASE STUDY 3: Trường hợp CÂN BẰNG (giả lập)\n")
pred, probs, recs = tactical_recommendation_system(balanced_stats, rf_model, scaler, win_thresholds)

## 10. TÓM TẮT VÀ KẾT LUẬN

In [ ]:
print("=" * 100)
print("📝 TÓM TẮT NGHIÊN CỨU")
print("=" * 100)

summary = f"""
1. DỮ LIỆU:
   • Số lượng: {len(df):,} trận đấu
   • Phân phối: Win {100*result_counts['Win']/len(df):.1f}%, Loss {100*result_counts['Loss']/len(df):.1f}%, Draw {100*result_counts['Draw']/len(df):.1f}%
   • Features ban đầu: {len(feature_columns)}
   • Features sau lựa chọn: {len(selected_features)}

2. PHƯƠNG PHÁP:
   • Feature Selection: ANOVA F-test + Correlation Analysis
   • Train/Test Split: 80/20 với Stratified Sampling
   • Chuẩn hóa: StandardScaler
   • Models: Naive Bayes (baseline) vs Random Forest

3. KẾT QUẢ:
   
   NAIVE BAYES:
   • Test Accuracy: {100*nb_test_acc:.2f}%
   • CV Accuracy: {100*cv_scores_nb.mean():.2f}% (±{100*cv_scores_nb.std():.2f}%)
   • Overfitting: {nb_train_acc - nb_test_acc:.4f}
   
   RANDOM FOREST:
   • Test Accuracy: {100*rf_test_acc:.2f}%
   • CV Accuracy: {100*cv_scores_rf.mean():.2f}% (±{100*cv_scores_rf.std():.2f}%)
   • Overfitting: {rf_train_acc - rf_test_acc:.4f}
   • Improvement: +{100*(rf_test_acc - nb_test_acc):.2f}%

4. FEATURE IMPORTANCE (Top 3):
   1. {feature_importance_df.iloc[0]['Feature']:20s}: {100*feature_importance_df.iloc[0]['Importance']:.1f}%
   2. {feature_importance_df.iloc[1]['Feature']:20s}: {100*feature_importance_df.iloc[1]['Importance']:.1f}%
   3. {feature_importance_df.iloc[2]['Feature']:20s}: {100*feature_importance_df.iloc[2]['Importance']:.1f}%

5. KẾT LUẬN:
   ✅ Random Forest vượt trội hơn Naive Bayes
   ✅ xG (Expected Goals) là yếu tố quyết định nhất
   ✅ Mô hình có khả năng dự đoán Win/Loss tốt (F1 > 0.60)
   ⚠️ Draw vẫn khó dự đoán (F1 ≈ 0.20)
   ✅ Hệ thống đề xuất chiến thuật hoạt động hiệu quả

6. ỨNG DỤNG:
   • Hỗ trợ HLV phân tích điểm mạnh/yếu của đội
   • Đề xuất chiến thuật cải thiện dựa trên data
   • Dự đoán kết quả trận đấu với độ tin cậy hợp lý
"""

print(summary)
print("=" * 100)

## 11. LƯU MÔ HÌNH

In [ ]:
import pickle

# Lưu mô hình Random Forest
with open('random_forest_model.pkl', 'wb') as f:
    pickle.dump(rf_model, f)

# Lưu scaler
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Lưu các thông tin cần thiết
model_info = {
    'selected_features': selected_features,
    'win_thresholds': win_thresholds,
    'feature_importance': feature_importance_df.to_dict(),
    'test_accuracy': rf_test_acc,
    'cv_scores': cv_scores_rf.tolist()
}

with open('model_info.pkl', 'wb') as f:
    pickle.dump(model_info, f)

print("✅ Đã lưu mô hình và thông tin!")
print("\nFiles đã lưu:")
print("  • random_forest_model.pkl")
print("  • scaler.pkl")
print("  • model_info.pkl")

---

## 🎉 HOÀN THÀNH!

**Tác giả:** [Tên của bạn]

**Ngày:** 2026-02-12

**Công cụ:** Python, scikit-learn, pandas, matplotlib, seaborn

**Liên hệ:** [Email của bạn]

---